<a href="https://colab.research.google.com/github/hida0/ESAA-2022-2/blob/HW/Week13_HW1_%EC%95%84%EC%9D%B4%ED%85%9C_%EA%B8%B0%EB%B0%98_%EC%B5%9C%EA%B7%BC%EC%A0%91_%EC%9D%B4%EC%9B%83_%ED%98%91%EC%97%85_%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 개념 필사
(파이썬 머신러닝 완벽가이드 ch9. 6, 7 pg. 591~606)

## 아이템 기반 최근접 이웃 협업 필터링 실습
- **최근접 이웃 협업 필터링**은 **사용자 기반과 아이템 기반**으로 분류한다.
- 일반적으로 **추천 정확도가 더 뛰어난 아이템 기반의 협업 필터링을 구현**한다.
- 협업 필터링 기반의 영화 추천을 위해서는 사용자가 영화의 평점을 매긴 사용자-영화 평점 행렬 데이터 세트가 필요하다.
- 이를 위해서 Grouplens 사이트의 MovieLens 데이터 세트를 이용하여 실습한다.

### 1. 데이터 가공 및 변환
- movies.csv 파일은 영화에 대한 메타 정보인 title과 genres를 가지고 있는 영화 정보이다.
- ratings.csv 파일은 사용자별로 영화에 대한 평점을 매긴 데이터 세트이다. 100836개의 레코드 세트로서 사용자 아이디를 의미하는 userId, 영화 아이디를 의미하는 movieId, 평점인 rating 칼럼으로 구성된다. 평점은 최소 0.5에서 최대 5점 사이이며, 0.5 단위로 부여된다.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
movies=pd.read_csv('/content/drive/MyDrive/ESAA/data/movies.csv')
print(movies.shape)
movies.head(3)

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [ ]:
ratings=pd.read_csv('/content/drive/MyDrive/ESAA/data/ratings.csv')
print(ratings.shape)
ratings.head(3)

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


### 1. 데이터 가공 및 변환
- **협업 필터링**은 이 ratings.csv 데이터 세트와 같이 **사용자와 아이템 간의 평점에 기반하여 추천하는 시스템**이다. ratings.csv의 DataFrame인 **rating를 이용하여 아이템 기반의 최근접 이웃 협업 필터링을 구현**한다.
- 먼저 로우 레벨 형태의 원본 데이터 세트를 **모든 사용자를 로우(행)**로, **모든 영화를 칼럼(열)**으로 구성한 데이터 세트로 변경해야 한다.
- 이와 같은 변환은 DataFrame의 **`pivot_table()`** 함수를 이용하여 쉽게 가능하다. **`pivot_table()`** 함수는 **로우 레벨의 값을 칼럼으로 변경**하는데 효과적이다. 
- **`pivot_table()`**의 인자로 **`columns='movieId'`**와 같이 부여하면 **movieId 칼럼의 모든 값이 새로운 칼럼 이름으로 변환**된다.
- `ratings.pivot_table('rating', index='userId', columns='movieId')`와 같이 호출하면 **로우 레벨은 userId**, **칼럼은 모두 movie 칼럼에 있는 값**으로 칼럼 이름이 바뀌고, **데이터는 rating 칼럼에 있는 평점 값**이 할당된다.


In [ ]:
ratings=ratings[['userId', 'movieId', 'rating']]
ratings_matrix=ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1. 데이터 가공 및 변환
- 모든 칼럼명이 movieId 값인 1부터 193609까지의 값으로 변환되었다.
- 칼럼명이 숫자로 되어 있으므로, 가독성을 높이기 위해 칼럼명을 movieId가 아닌 영화명 title로 변경한다.
- 영화명은 movies 데이터 세트에 존재하므로 판다스의 **`merge`** 함수의 인자로 **`on='movieId'`**를 입력하여 ratings와 movies를 'movieId' 기준으로 조인하여 title 칼럼을 가져온다.
-  **`pivot_table()`**의 인자로 **`columns='title'`**을 입력하여 title로 pivot한다.
- NaN 값이 많은데, 그 이유는 사용자가 평점을 매기지 않은 영화가 칼럼으로 변환되면서 NaN 값이 할당되었기 때문이다. 최소 평점이 0.5이므로 **NaN을 모두 0으로 변환**한다.


In [ ]:
#title 칼럼을 얻기 위해 movies와 조인
rating_movies=pd.merge(ratings, movies, on='movieId')
rating_movies.tail()

,userId,movieId,rating,title,genres
100831,610,160341,2.5,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,Blair Witch (2016),Horror|Thriller
100835,610,163981,3.5,31 (2016),Horror


In [ ]:
#columns='title'로 title 칼럼으로 피벗 수행
ratings_matrix=rating_movies.pivot_table('rating', index='userId', columns='title')

#NaN 값을 모두 0으로 변환
ratings_matrix=ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. 영화 간 유사도 산출
- 변환된 **사용자-영화 평점 행렬 데이터 세트**를 이용하여 **영화 간의 유사도를 측정**한다.
- **영화 간의 유사도는 코사인 유사도**를 기반으로 하고, 사이킷런의 **`cosine_similarity()`**를 **이용하여 측정**한다.
- 그런데 지금 만든 ratings_matrix 데이터 세트에 **`cosine_similarity()`**를 적용하면 영화간 유사도를 측정할 수 없다. 
- **`cosine_similarity()`**는 **행을 기준으로 서로 다른 행을 비교하여 유사도를 산출**하는데, 지금 ratings_matrix는 **userId가 기준인 행 레벨 데이터**이므로 **`cosine_similarity()`**를 적용하면 영화 간의 유사도가 아닌 **사용자 간의 유사도**가 만들어진다.
- 따라서 영화를 기준으로 **`cosine_similarity()`**를 적용하려면 현재의 ratings_matrix가 **행 기준이 영화**가 되고, **열 기준이 사용자**가 되야 한다.
- ratings_matrix 데이터의 **행과 열 위치를 변경**하기 위해 판다스에서 **전치 행렬 변경**을 위해 제공하는 **`transpose()`** 함수를 사용한다.


In [ ]:
ratings_matrix_T=ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. 영화 간 유사도 산출
- ratings_matrix를 전치 행렬 형식으로 변경한 데이터 세트를 기반으로 영화의 코사인 유사도를 계산한다. 
- 좀 더 직관적으로 영화의 유사도 값을 표현하기 위해 **`cosine_similarity()`**로 **반환된 넘파이 행렬에 영화명을 매핑**하여 DataFrame으로 변환한다.
- 9719 로우인 ratings_matrix_T 데이터 세트에 `cosine_similarity()`을 적용한 결과, 9719x9719 shape으로 영화의 유사도 행렬인 item_sim이 생성되었다.
---
- DataFrame으로 변환한 item_sim_df를 이용하여 영화 '대부(Godfathe, The (1972))'와 유사도가 높은 상위 6개 영화를 추출한다.
- 기준 영화인 'Godfather, The (1972)'를 제외하면 'Godfather: Part2, The (1974)'가 가장 유사도가 높다. 다음으로 'Goodfellas'가 유사도가 높다.
- 콘텐츠 기반 필터링과 다른 점은 'One Flew Over the Cuckoo's Nest (1975)', 'Star Wars: Episode IV - A New Hope'와 같이 **장르가 완전히 다른 영화도 유사도가 매우 높게 나타났다**는 점이다. 
--- 
- 다른 영화인 '인셉션(Inception (2010)'과 유사도가 높은 영화를 찾는다. 자기 자신은 유사도에서 제외한다. 
- 'Dark Knight, The (2008)'이 가장 유사도가 높다. 그 뒤를 이어서 주로 스릴러와 액션이 가미된 좋은 영화가 높은 유사도를 나타낸다. 
- **아이템 기반 유사도 데이터**는 **사용자의 평점 정보를 모두 취합**하여 **영화에 따라 유사한 다른 영화를 추천**할 수 있도록 한다.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
item_sim=cosine_similarity(ratings_matrix_T, ratings_matrix_T)

#cosine_similarity()로 반환된 넘파이 행렬에 영화명을 매핑하여 DataFrame으로 변환
item_sim_df=pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head()

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot (2004),0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You (1997),0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [ ]:
item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

In [ ]:
item_sim_df['Inception (2010)'].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

### 3. 아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천
- 영화 유사도 데이터를 이용하여 최근접 이웃 협업 필터링으로 개인에게 최적화된 영화 추천을 구현한다.
- **개인화된 영화 추천**의 가장 큰 특징은 **아직 관람하지 않은 영화를 추천**한다는 것이다. **아직 관람하지 않은 영화**에 대해서 **아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기반**으로 해 새롭게 **모든 영화의 예측 평점을 계산**한 후, **높은 예측 평점을 가진 영화를 추천**하는 방식이다.
- 아이템 기반의 협업 필터링에서 **개인화된 예측 평점**은 다음 식으로 구할 수 있다.
$\hat{R_{u,i}}=\sum^{n}(S_{i,N}*R_{u,N})/\sum^{N}(|S_{i,N}|)$
 - $N$: 아이템의 최근접 이웃 범위 계수
 - $\hat{R_{u,i}}$: 사용자 u, 아이템 i의 개인화된 예측 평점
 - $S_{i,N}$: 아이템 i와 가장 유사도가 높은 Top-N개 아이템의 유사도 벡터
 - $R_{u,N}$: 사용자 u의 아이템 i와 가장 유사도가 높은 Top-N개 아이템에 대한 실제 평점 벡터
---
- 먼저 $N$의 범위에 제약을 두지 않고 모든 아이템으로 가정하고 예측 평점을 구하는 로직을 작성한 뒤에, Top-N 아이템을 기반으로 협업 필터링을 수행하는 로직으로 변경한다.
- **영화 간의 유사도**를 가지는 DataFrame인 **item_sim_df**와 **사용자-영화 평**점 DataFrame인 **ratings_matrix** 변수를 계속 활용하여 **사용자별로 최적화된 평점 스코어를 예측하는 함수를 생성**한다.
- 함수 명은 **`predict_rating()`**이며, 인자로 사용자-영화 평점 넘파이 행렬(ratings_matrix를 넘파이 행렬로 변환)과 영화 간의 유사도를 가지는 넘파이 행렬(item_sim_df를 넘파이 행렬로 변환)을 입력받는다.
- 이를 이용하여 앞서 언급한 개인화된 예측 평점을 구하는 식으로 계산한다. $N$의 범위에 제약을 두지 않는다면 **사용자별 영화 예측 평점** $\hat{R_{u,i}}$는 **사용자 u의 모든 영화에 대한 실제 평점과 영화 i의 다른 모든 영화와의 코사인 유사도를 벡터 내적 곱한 값**을 정규화를 위해 **$\sum^{N}(|S_{i,N}|)$로 나눈 것**을 의미한다.
- 다음의 코드는 이를 구현한 것이다.

In [ ]:
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred=ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

### 3. 아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천
- predict_rating() 함수를 이용해 개인화된 예측 평점을 구한다. 이를 위해 ratings_matrix와 item_sim_df를 넘파이 행렬로 변환하여 인자로 입력한다.
- 예측 평점이 사용자별 영화의 실제 평점과 영화의 코사인 유사도를 내적한 값이기 때문에 기존에 영화를 관람하지 않아 0에 해당했던 실제 영화 평점이 예측에서는 값이 부여되는 경우가 많이 발생한다. 
---
- 이렇게 구한 **예측 결과가 원래의 실제 평점과 얼마나 차이가 있는지** 확인하기 위해 **예측 평가 지표 MSE**를 이용한다.
- 단, MSE를 측정할 때 유의할 점이 있다. 사용자가 영화의 평점을 주지 않은 경우 앞에서는 평점을 0으로 부과했다. 하지만 개인화된 예측 점수는 평점을 주지 않은 영화에 대해서도 아이템 유사도에 기반하여 평점을 예측했다.
- 따라서 **실제와 예측 평점의 차이는 기존에 평점이 부여된 데이**터에 대해서만 오차 정도를 측정한다. 
- 예측 평가 지표인 MSE를 계산하는 **`get_mse()`** 함수를 만들고 결과를 확인한다.
- MSE가 약 9.89이다.

In [ ]:
ratings_pred=predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix=pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501
4,0.049145,0.277628,0.160448,0.206892,0.309632,0.042337,0.130048,0.116442,0.099785,0.097432,...,0.051269,0.076051,0.055563,0.054137,0.008343,0.159242,0.100941,0.062253,0.146054,0.231187
5,0.007278,0.066951,0.041879,0.013880,0.024842,0.018240,0.026405,0.018673,0.021591,0.018841,...,0.009689,0.022246,0.013360,0.012378,0.000000,0.025839,0.023712,0.018012,0.028133,0.052315


In [ ]:
from sklearn.metrics import mean_squared_error
#사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함
def get_mse(pred, actual):
    #평점이 있는 실제 영화만 추출
    pred=pred[actual.nonzero()].flatten()
    actual=actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print("아이템 기반 모든 최근접 이웃 MSE: ", get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE:  9.895354759094706


### 3. 아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천
- `predict_rating()` 함수는 사용자별 영화의 예측 평점을 계산하기 위해 해당 영화와 다른 모든 영화 간의 유사도 벡터를 적용한 것이다. 많은 영화의 유사도 벡터를 이용하다보니 상대적으로 평점 예측이 떨어졌다.
- 따라서 특정 영화와 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터를 적용하는 함수로 변경한다. **`predict_rating_topsim(ratings_arr, item_sim_arr, N=20)`** 함수는 `predict_rating()`와 유사하지만 N 인자를 가지고 있어서, **Top-N 유사도를 가지는 영화 유사도 벡터만 예측값을 계산하는데 적용**한다.
- **`predict_rating_topsim()`** 함수를 이용하여 예측 평점을 계산하고, 실제 평점과의 MSE를 구한다. 
- 계산된 예측 평점 넘파이 행렬은 판다스 DataFrame으로 재생성한다.
- MSE가 3.69로, 기존의 9.89보다 많이 향상되었다.

In [ ]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    #사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred=np.zeros(ratings_arr.shape)

    #사용자-아이템 평점 행렬의 열 크기만큼 루프 수행
    for col in range(ratings_arr.shape[1]):
        #유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        top_n_items=[np.argsort(item_sim_arr[:, col])[:(-n-1):-1]]
        #개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col]=item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
            pred[row, col]/=np.sum(np.abs(item_sim_arr[col, :][top_n_items]))

    return pred

In [ ]:
ratings_pred=predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print("아이템 기반 최근접 TOP-20 이웃 MSE: ", get_mse(ratings_pred, ratings_matrix.values))

#계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix=pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

아이템 기반 최근접 TOP-20 이웃 MSE:  3.6949827608772314


### 3. 아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천
- 특정 사용자에 대해 영화를 추천한다.
- `userId=9`인 사용자에 대해 영화를 추천하기 위해, 먼저 9번 사용자가 어떤 영화를 좋아하는지 확인한다. 이를 위해 사용자가 평점을 준 영화를 평점이 높은 순으로 나열한다.
- 이 사용자에게 아이템 기반 협업 필터링을 통해 영화를 추천한다.
- 먼저 사용자가 이미 평점을 준 영화를 제외하고 추천할 수 있도록 평점을 주지 않은 영화를 리스트 객체로 반환하는 함수인 **`get_unseen_movies()`**를 생성한다.
- 사용자가 영화의 평점을 주지 않은 추천 대상 영화 정보와 `predict_rating_topsim()`에서 추출한 사용자별 아이템 유사도에 기반한 예측 평점 데이터 세트를 이용하여 최종적으로 사용자에게 영화를 추천하는 함수인 **`recomm_movie_by_userid()`**를 생성한다.
- **`recomm_movie_by_userid()`**는 예측 평점 DataFrame과 추천하려는 userId, 아직 평점을 주지 않은 추천 후보 영화 리스트, 추천 상위 영화 개수를 인자로 받아서 사용자가 좋아할 만한 가장 높은 예측 평점을 가진 영화를 추천한다.

In [ ]:
user_rating_id=ratings_matrix.loc[9, :]
user_rating_id[user_rating_id>0].sort_values(ascending=False)[:10]

title
Adaptation (2002)                                                                 5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Producers, The (1968)                                                             5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Back to the Future (1985)                                                         5.0
Austin Powers in Goldmember (2002)                                                5.0
Minority Report (2002)                                                            4.0
Witness (1985)                                                                    4.0
Name: 9, dtype: float64

In [ ]:
def get_unseen_movies(ratings_matrix, userId):
    #userId로 입력받은 사용자의 모든 영화 정보를 추출하여 영화명을 인덱스로 가지는 Series 객체인 user_rating로 반환
    user_rating=ratings_matrix.loc[userId, :]
    #user_rating이 0보다 크면 기존에 관람한 영화로, 대상 인덱스를 추출하여 list 객체로 만듦
    already_seen=user_rating[user_rating>0].index.tolist()
    #모든 영화명을 list 객체로 만듦
    movies_list=ratings_matrix.columns.tolist()
    #list comprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함
    unseen_list=[movie for movie in movies_list if movie not in already_seen]
    return unseen_list

In [ ]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    #예측 평점 DataFrame에서 userId 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출하여 가장 예측 평점이 높은 순으로 정렬
    recomm_movies=pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [ ]:
#사용자가 관람하지 않은 영화명 추출
unseen_list=get_unseen_movies(ratings_matrix, userId=9)

#아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies=recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

#평점 데이터를 DataFrame으로 생성
recomm_movies=pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


## 행렬 분해를 이용한 잠재 요인 협업 필터링 실습
- 행렬 분해 잠재 요인 협업 필터링은 SVD나 NMF 등을 적용할 수 있다. 일반적으로 행렬 분해에는 SVD가 자주 사용되지만, **사용자-아이템 평점 행렬에는 사용자가 평점을 매기지 않은 NaN 데이터가 많기 때문**에 **주로 SGD나 ALS 기반의 행렬 분해를 이용**한다.
- 앞의 잠재 요인 협업 필터링 절의 확률적 경사 하강법을 이용한 행렬 분해에서 사용한 예제 코드를 다시 활용한다.
- 확률적 경사 하강법을 이용한 행렬 분해 예제의 **`get_rmse()`** 함수를 사용하고, 행렬 분해 로직을 새로운 **`matrix_factorization()`** 함수로 정리한다.
- **`matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01)`**에서 R은 원본 사용자-아이템 평점 행렬, K는 잠재 요인의 차원 수, steps는 SGD의 반복 횟수, learning_rate는 학습률, r_lambda는 L2 규제 계수이다.
- 영화 평점 행렬 데이터를 새롭게 DataFrame으로 로딩한 뒤에, 다시 사용자-아이템 평점 행렬로 만든다.
- 다시 만들어진 사용자-아이템 평점 행렬을 `matrix_factorization()` 함수를 이용하여 행렬 분해 한다. 

In [ ]:
from sklearn.metrics import mean_squared_error
def get_rmse(R, P, Q, non_zeros):
    error=0
    #2개의 분해된 행렬 P와 Q.T의 내적으로 예측 행렬 R 생성
    full_pred_matrix=np.dot(P, Q.T)

    #실제 R 행렬에서 NaN이 아닌 값의 위치 인덱스를 추출하여 실제 R 행렬과 예측 행렬의 RMSE를 추출
    x_non_zero_ind=[non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind=[non_zero[1] for non_zero in non_zeros]
    R_non_zeros=R[x_non_zero_ind, y_non_zero_ind] #실제 R 행렬
    full_pred_matrix_non_zeros=full_pred_matrix[x_non_zero_ind, y_non_zero_ind] #예측 행렬
    mse=mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse=np.sqrt(mse)
    
    return rmse

In [ ]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items=R.shape
    #P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P=np.random.normal(scale=1./K, size=(num_users, K))
    Q=np.random.normal(scale=1./K, size=(num_items, K))
    prev_rmse=10000
    break_count=0

    #R>0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
    non_zeros=[(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0]

    #SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            #실제값과 예측값의 차이인 오류값을 구함
            eij=r-np.dot(P[i, :], Q[j, :].T)
            #Regularization을 반영한 SGD 업데이트 공식 적용
            P[i, :]=P[i, :]+learning_rate*(eij*Q[j, :]-r_lambda*P[i, :])
            Q[j, :]=Q[j, :]+learning_rate*(eij*P[i, :]-r_lambda*Q[j, :])

        rmse=get_rmse(R, P, Q, non_zeros)
        if (step%10)==0:
            print("### iteration step : ", step, " RMSE: ", rmse)

    return P, Q

In [ ]:
movies=pd.read_csv('/content/drive/MyDrive/ESAA/data/movies.csv')
ratings=pd.read_csv('/content/drive/MyDrive/ESAA/data/ratings.csv')
ratings=ratings[['userId', 'movieId', 'rating']]
ratings_matrix=ratings.pivot_table('rating', index='userId', columns='movieId')

#title 칼럼을 얻기 위해 movies와 merge 함수를 이용하여 join 수행
rating_movies=pd.merge(ratings, movies, on='movieId')
#columns='title'로 title 칼럼으로 피벗 수행
ratings_matrix=rating_movies.pivot_table('rating', index='userId', columns='title')

In [ ]:
P, Q=matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix=np.dot(P, Q.T)

### iteration step :  0  RMSE:  2.9023619751336867
### iteration step :  10  RMSE:  0.7335768591017927
### iteration step :  20  RMSE:  0.5115539026853442
### iteration step :  30  RMSE:  0.37261628282537446
### iteration step :  40  RMSE:  0.2960818299181014
### iteration step :  50  RMSE:  0.2520353192341642
### iteration step :  60  RMSE:  0.22487503275269854
### iteration step :  70  RMSE:  0.2068545530233154
### iteration step :  80  RMSE:  0.19413418783028685
### iteration step :  90  RMSE:  0.18470082002720403
### iteration step :  100  RMSE:  0.17742927527209104
### iteration step :  110  RMSE:  0.17165226964707492
### iteration step :  120  RMSE:  0.16695181946871723
### iteration step :  130  RMSE:  0.16305292191997545
### iteration step :  140  RMSE:  0.15976691929679646
### iteration step :  150  RMSE:  0.1569598699945732
### iteration step :  160  RMSE:  0.1545339818671543
### iteration step :  170  RMSE:  0.15241618551077643
### iteration step :  180  RMSE:  0.15055080739

## 행렬 분해를 이용한 잠재 요인 협업 필터링 실습
- 반환된 예측 사용자 평점 행렬을 영화 타이틀을 칼럼으로 가지는 DataFrame으로 변경한다.
- 이렇게 만들어진 예측 사용자-아이템 평점 행렬 정보를 이용하여 **개인화된 영화 추천**을 한다.
- 아이템 기반 최근접 이웃 협업 필터링 실습과 동일하게 userId=9인 **사용자에 대한 영화 추천을 잠재 요인 협업 필터링으로 추천**한다.
- 앞에서 만든 **`get_unseen_movies()`** 함수와 **`recomm_movie_by_userid()`** 함수를 다시 이용하여 추천 영화를 추출한다.

In [ ]:
ratings_pred_matrix=pd.DataFrame(data=pred_matrix, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941
4,2.628629,3.035550,2.575746,3.706912,3.430636,0.706441,3.330280,1.978826,4.560368,2.775710,...,1.046116,2.912178,2.479592,2.231915,1.888629,2.211364,0.645603,1.585734,3.542892,0.591540
5,2.116148,3.084761,2.747679,3.783490,3.946990,0.883259,1.958953,1.757317,2.054312,2.775258,...,0.956159,3.893975,2.717024,2.002443,2.053337,3.983639,2.099626,1.423718,2.490428,0.531403


In [ ]:
#사용자가 관람하지 않은 영화명을 추출
unseen_list=get_unseen_movies(ratings_matrix, userId=9)

#잠재 요인 협업 필터링으로 영화 추천
recomm_movies=recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

#평점 데이터를 DataFrame으로 생성
recomm_movies=pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690
